# Visualize blobs (nuclei) found in julia and remove misslabeld ones

This notebook aids filtering-out and manually selecting mis-detected nuclei

In [1]:
import napari
import numpy as np
import random
%gui qt
import sys
import yaml
sys.path.append('..')
from calcium_roi_analysis.io import load_tiffstack
from calcium_roi_analysis.params import prepare_paths_3d

## Prepare paths

⚠️ Most inputs come from file "../current.yaml". Place a `current.yaml` file in the parent directory of this notebook. See example files in the repository. The only variable controlled here is **review_only**

In [2]:
review_only = False

In [3]:
params, data_path, output_path = prepare_paths_3d("../parameters/current.yaml")

## Load pre-saved volumes of interest

In [4]:
voi = load_tiffstack(data_path)

/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs  --  ['/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z001_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z002_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z003_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z004_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z005_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z006_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z007_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs/Microglia_Sample_2channel_z008_c001.tif', '/Users/mrestrep/projects/borton/calacium_images/sophie_

In [5]:
blob_sigmas = [round(x) for x in params["radius"]/np.sqrt(2.)]
sigma_string = "-".join(map(str,blob_sigmas)).replace(".", "_")

## Load pre-combuted Julia blobs

In [6]:
blobs_file = output_path + f"/blobs-data-{ sigma_string }.npz"
all_blobs_data = np.load(blobs_file)

In [7]:
points = all_blobs_data['coords']
class_labels = all_blobs_data['class_label']  
confidence = all_blobs_data['confidence']
properties = {
    'confidence': confidence,
    'good_point': class_labels
}
confidence_t = float(all_blobs_data['threshold'])

In [8]:
sigma = all_blobs_data['sigma']
# min_amplitude = min(all_blobs[:, 3])
# max_amplitude = max(all_blobs[:, 3])
# intensity = (all_blobs[:, 3]-min_amplitude)/max_amplitude
# intensity = intensity.reshape((all_blobs[:, 3].shape[0], 1))
# points = np.concatenate((intensity, all_blobs[roi_idx][:, 0:3]), axis=1)

In [9]:
confidence

array([ 1.94969042e-06,  4.48597694e-08,  1.98018343e-06, ...,
       -5.98148776e-07, -2.57177409e-06,  4.65723868e-06])

# Adjust class labels

1. Use key bindings to increase and decrease threshold:
    * y +0.0001; u +0.001; i + 0.01
    * t -0.0001; r -0.001; e - 0.01
2. Toggle class of points by selecting them and pressing `c` 
3. You can add and remove points using Napari's lateer tools
4. Once you are satisfied press `f` to save results to file
5. You can press `h` to toggle show/hide bad(blue) points

In [10]:
if not review_only:
    viewer = napari.view_image(voi, rgb=False)

    # define the color cycle for the face_color annotation
    face_color_cycle = [[0, 0, 1, 0.5], [0, 1, 0, 0.5]]
    show_bad_points=True

    points_layer = viewer.add_points(points,
                        face_color='good_point',
                        face_color_cycle=face_color_cycle,
                        edge_color=[1, 1, 1, 0],
                        name='all_points',
                        size=sigma,
                        properties=properties)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('y')
    def increase_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t + 0.0001
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('t')
    def decrease_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t -  0.0001
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('u')
    def increase_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t + 0.001
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('r')
    def decrease_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t -  0.001
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('i')
    def increase_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t + 0.01
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('e')
    def decrease_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t -  0.01
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    @viewer.bind_key('d')
    def decrease_threshold(viewer):
        print("bye")
        global confidence_t
        print(confidence_t)
        confidence_t  = confidence_t -  0.01
        points_layer.properties['good_point'] = points_layer.properties['confidence'] > confidence_t

        # we need to manually refresh since we did not use the Points.properties setter
        # to avoid changing the color map if all points get toggled to the same class,
        # we set update_colors=False (only re-colors the point using the previously-determined color mapping).
        points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to toggle the good_point annotation of the selected points
    @viewer.bind_key('c')
    def toggle_class(viewer):
        selected_points = np.array(list(points_layer.selected_data))
        if len(selected_points) > 0:
            print(f"Toggling {len(selected_points)} points")
            good_point = points_layer.properties['good_point']
            good_point[selected_points] = ~good_point[selected_points]
            points_layer.properties['good_point'] = good_point
            points_layer.refresh_colors(update_color_mapping=False)

    # bind a function to hide the bad (blue - points)
    @viewer.bind_key('h')
    def hide_bad_points(viewer):
        global show_bad_points
        if show_bad_points:
            points_layer.face_color_cycle = [[0, 0, 1, 0], [0, 1, 0, 1]]
            show_bad_points = False
        else:
            points_layer.face_color_cycle = [[0, 0, 1, 1], [0, 1, 0, 1]]
            show_bad_points = True
            
            
    @viewer.bind_key('f')
    def save_points(viewer):

        print("Saving labeled points to file")
        #save results to files.

        fout = output_path + "/curated_blobs.npz"
        np.savez(fout, coords=points_layer.data, class_label=points_layer.properties['good_point'], 
                       size=points_layer.size, confidence=points_layer.properties['confidence'],
                       threshold=[confidence_t])

        print("Saved")


## Reload and visualize class labels for verification

You can still toggle class, add/remove points using napari's point-layer interface and resave

In [51]:
curated_blobs_file = output_path  + "/curated_blobs.npz"
curated_roi = np.load(curated_blobs_file)
review_points = curated_roi['coords']
class_labels = curated_roi['class_label']    
review_properties = {
    'confidence': curated_roi['confidence'],
    'good_point': class_labels
}
confidence_t = curated_roi['threshold'][0]


In [53]:
# define the color cycle for the face_color annotation
show_bad_points=True
face_color_cycle = [[0, 0, 1, 0.5], [0, 1, 0, 0.5]]

review_viewer = napari.view_image(voi, rgb=False)


review_layer = review_viewer.add_points(review_points,
                    face_color='good_point',
                    edge_color=[1, 1, 1, 0],
                    face_color_cycle=face_color_cycle,
                    name='all points',
                    size=3,
                    properties=review_properties)


# bind a function to toggle the good_point annotation of the selected points
@review_viewer.bind_key('c')
def toggle_class(review_viewer):
    selected_points = np.array(list(review_layer.selected_data))
    if len(selected_points) > 0:
        print(f"Toggling {len(selected_points)} points")
        good_point = review_layer.properties['good_point']
        good_point[selected_points] = ~good_point[selected_points]
        review_layer.properties['good_point'] = good_point
        review_layer.refresh_colors(update_color_mapping=False)


# bind a function to hide the bad (blue - points)
@review_viewer.bind_key('h')
def hide_bad_points(review_viewer):
    global show_bad_points
    if show_bad_points:
        review_layer.face_color_cycle = [[0, 0, 1, 0], [0, 1, 0, 1]]
        show_bad_points = False
    else:
        review_layer.face_color_cycle = [[0, 0, 1, 1], [0, 1, 0, 1]]
        show_bad_points = True
        
@review_viewer.bind_key('f')
def save_points(review_viewer):

    print("Saving labeled points to file")
    #save results to files.

    fout = output_path + "/roi_" + str(roi_idx) + "/curated_blobs.npz"
    np.savez(fout, coords=review_layer.data, class_label=review_layer.properties['good_point'], 
                    size=review_layer.size, confidence=review_layer.properties['confidence'],
                    threshold=[confidence_t])

    print("Saved")
